# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
conn.autocommit = True
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
filepath = song_files[0]
print(f"filepath: {filepath}")

filepath: /sourcecode/verna/gitlab/tutorials/udacity-data-engineering/projects/1_data-modeling-postgres/data/song_data/A/A/C/TRAACCG128F92E8A55.json


In [6]:
df = pd.read_json(filepath, lines=True)
df.head(1)

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR5KOSW1187FB35FF4,49.80388,15.47491,Dubai UAE,Elena,SOZCTXZ12AB0182364,Setanta matins,269.58322,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = [df.song_id[0], df.title[0], df.artist_id[0], df.year.values[0].astype(int).item(), df.duration[0]]
# song_data = [df.song_id[0], df.title[0], df.artist_id[0], df.year[0], df.duration[0]]
# song_data = df.loc[:]
# song_data = df['song_id', 'title', 'artist_id', 'year', 'duration']
# song_data = df['song_id'], df['title'], df['artist_id'], df['year'], df['duration']
# song_data = [list(row) for row in df.itertuples(index=False)] 
type(song_data[3])
# song_data

int

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
print(f"query: {song_table_insert}")
cur.execute(song_table_insert, song_data)
conn.commit()

query: 
insert into songs values (%s, %s, %s, %s, %s)



Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = [df.artist_id[0], df.artist_name[0], df.artist_location[0], df.artist_latitude[0], df.artist_longitude[0]]
artist_data

['AR5KOSW1187FB35FF4', 'Elena', 'Dubai UAE', 49.80388, 15.47491]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files("data/log_data")

In [12]:
filepath = log_files[2]
filepath

'/sourcecode/verna/gitlab/tutorials/udacity-data-engineering/projects/1_data-modeling-postgres/data/log_data/2018/11/2018-11-25-events.json'

In [13]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,matchbox twenty,Logged In,Jayden,F,0,Duffy,177.65832,free,"Seattle-Tacoma-Bellevue, WA",PUT,NextSong,1.540146e+12,846,Argue (LP Version),200,1543109954796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",76
1,The Lonely Island / T-Pain,Logged In,Jayden,F,1,Duffy,156.23791,free,"Seattle-Tacoma-Bellevue, WA",PUT,NextSong,1.540146e+12,846,I'm On A Boat,200,1543110131796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",76
2,None,Logged In,Jayden,F,2,Duffy,NaN,free,"Seattle-Tacoma-Bellevue, WA",GET,Home,1.540146e+12,846,None,200,1543110132796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",76
3,None,Logged In,Jayden,F,3,Duffy,NaN,free,"Seattle-Tacoma-Bellevue, WA",GET,Settings,1.540146e+12,846,None,200,1543110168796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",76
4,None,Logged In,Jayden,F,4,Duffy,NaN,free,"Seattle-Tacoma-Bellevue, WA",PUT,Save Settings,1.540146e+12,846,None,307,1543110169796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",76


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df = pd.read_json(filepath, lines=True)
df = df[df['page']=='NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,matchbox twenty,Logged In,Jayden,F,0,Duffy,177.65832,free,"Seattle-Tacoma-Bellevue, WA",PUT,NextSong,1.540146e+12,846,Argue (LP Version),200,1543109954796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",76
1,The Lonely Island / T-Pain,Logged In,Jayden,F,1,Duffy,156.23791,free,"Seattle-Tacoma-Bellevue, WA",PUT,NextSong,1.540146e+12,846,I'm On A Boat,200,1543110131796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",76
5,John Mayer,Logged In,Wyatt,M,0,Scott,275.27791,free,"Eureka-Arcata-Fortuna, CA",PUT,NextSong,1.540872e+12,856,All We Ever Do Is Say Goodbye,200,1543113347796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9
7,10_000 Maniacs,Logged In,Wyatt,M,2,Scott,251.87220,free,"Eureka-Arcata-Fortuna, CA",PUT,NextSong,1.540872e+12,856,Gun Shy (LP Version),200,1543113622796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9
8,Leona Lewis,Logged In,Chloe,F,0,Cuevas,203.88526,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,916,Forgive Me,200,1543122348796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49


In [15]:
t = df['ts']
t = pd.to_datetime(t, unit='ms')
t.head()

# print(t.dt.hour.values[0:5])

0   2018-11-25 01:39:14.796
1   2018-11-25 01:42:11.796
5   2018-11-25 02:35:47.796
7   2018-11-25 02:40:22.796
8   2018-11-25 05:05:48.796
Name: ts, dtype: datetime64[ns]

In [50]:
# Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column
# and set `time_data` to a list containing these values in order

time_data = (t.values[0], t.dt.hour.values[0], t.dt.day.values[0],
             t.dt.isocalendar().week.values[0], t.dt.month.values[0],
             t.dt.year.values[0], t.dt.weekday.values[0])

# time_data = (t.values, t.dt.hour.values, t.dt.day.values,
#              t.dt.isocalendar().week.values, t.dt.month.values,
#              t.dt.year.values, t.dt.weekday.values)


print(time_data)
print(f"time_data type: {type(time_data)}")

#Specify labels for these columns and set to `column_labels`
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')
# type(column_labels)

(numpy.datetime64('2018-11-25T01:39:14.796000000'), 1, 25, 47, 11, 2018, 6)
time_data type: <class 'tuple'>


In [68]:
time_df = pd.DataFrame([time_data], columns=column_labels)
print(f"rows: {time_df.count()[0]}")
time_df

rows: 1


,start_time,hour,day,week,month,year,weekday
0,2018-11-25 01:39:14.796,1,25,47,11,2018,6


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = 

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.